Accordion(children=(Tab(children=(VBox(children=(HTML(value='<h4>Secure Login / Logout</h4>'), IntText(value=0…

In [3]:
import pickle, os, requests, hashlib, csv, random
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink, HTML, IFrame
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np
import seaborn as sns
import pandas as pd
import plotly.express as px

# ML libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report
import joblib
import xgboost as xgb

# For modern interactive mapping
import folium

###############################################################################
# HELPER FUNCTIONS & CONFIG
###############################################################################

def fetch_live_conversion_rates(base_currency="USD"):
    try:
        response = requests.get(f"https://api.exchangerate-api.com/v4/latest/{base_currency}")
        if response.status_code == 200:
            data = response.json()
            rates = data.get("rates", {})
            rates["Local"] = 1.0
            return rates
        else:
            print("Error fetching live rates. Using fallback rates.")
            return {"Local": 1.0, "USD": 1.0, "EUR": 1.1, "GBP": 1.2}
    except Exception as e:
        print("Exception fetching live rates:", e)
        return {"Local": 1.0, "USD": 1.0, "EUR": 1.1, "GBP": 1.2}

# Secure password hashing
def hash_password(password):
    salt = os.urandom(16)
    pwdhash = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000)
    return salt + pwdhash

def check_password(password, hashed):
    salt = hashed[:16]
    stored_pwdhash = hashed[16:]
    pwdhash = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000)
    return pwdhash == stored_pwdhash

# Global variables for currency rates & user state
conversion_rates = fetch_live_conversion_rates("USD")
current_user = None

# Security / OTP / Rate Limiting
failed_logins = {}  # {accNo: (fail_count, last_fail_datetime)}
otp_data = {}       # {accNo: (otp_code, expiry_datetime)}
MAX_FAILED_ATTEMPTS = 3
LOCKOUT_DURATION = timedelta(minutes=5)
OTP_VALIDITY_DURATION = timedelta(minutes=2)

###############################################################################
# DATA MODELS
###############################################################################
class Account:
    def __init__(self, accNo, name, acc_type, deposit, monthly_income,
                 hashed_password, role, email, phone, address, dob):
        self.accNo = accNo
        self.name = name
        self.acc_type = acc_type
        self.deposit = deposit
        self.monthly_income = monthly_income
        self.password = hashed_password
        self.role = role
        self.email = email
        self.phone = phone
        self.address = address
        self.dob = dob

        self.transaction_history = []
        self.loan_details = None
        self.kyc_docs = []
        self.kyc_status = "None"  # "None", "Under Review", "Approved", "Rejected"
        self.notification_pref = "Email"
        self.badges = []
        self.add_transaction("Initial Deposit", deposit, "Account creation deposit")

    def add_transaction(self, trans_type, amount, description):
        txn = {
            'transaction_id': str(os.urandom(4).hex()),
            'type': trans_type,
            'amount': amount,
            'description': description,
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        self.transaction_history.append(txn)

    def __str__(self):
        return (f"Acc No: {self.accNo} | Name: {self.name} | Type: {self.acc_type} | "
                f"Balance: {self.deposit} | Income: {self.monthly_income} | "
                f"Role: {self.role} | Email: {self.email} | Phone: {self.phone} | "
                f"Address: {self.address} | DOB: {self.dob} | KYC: {self.kyc_status}")

class BankSystem:
    def __init__(self, data_file="accounts.data"):
        self.data_file = Path(data_file)
        self.accounts = []
        self.load_accounts()

    def load_accounts(self):
        if self.data_file.exists():
            try:
                with open(self.data_file, 'rb') as f:
                    self.accounts = pickle.load(f)
            except Exception as e:
                print("Error loading accounts:", e)
                self.accounts = []
        else:
            self.accounts = []

    def save_accounts(self):
        with open(self.data_file, 'wb') as f:
            pickle.dump(self.accounts, f)

    def add_account(self, account):
        if self.get_account(account.accNo) is not None:
            return False
        self.accounts.append(account)
        self.save_accounts()
        return True

    def get_account(self, accNo):
        for acc in self.accounts:
            if acc.accNo == accNo:
                return acc
        return None

    def deposit(self, accNo, amount):
        account = self.get_account(accNo)
        if account:
            account.deposit += amount
            account.add_transaction("Deposit", amount, "Amount deposited")
            self.save_accounts()
            return account
        return None

    def withdraw(self, accNo, amount):
        account = self.get_account(accNo)
        if account:
            if account.deposit >= amount:
                account.deposit -= amount
                account.add_transaction("Withdrawal", amount, "Amount withdrawn")
                self.save_accounts()
                return account
            else:
                return "Insufficient Funds"
        return None

    def delete_account(self, accNo):
        account = self.get_account(accNo)
        if account:
            self.accounts.remove(account)
            self.save_accounts()
            return True
        return False

    def modify_account(self, accNo, name, acc_type, deposit, monthly_income, email, phone, address, dob):
        account = self.get_account(accNo)
        if account:
            account.name = name
            account.acc_type = acc_type
            account.deposit = deposit
            account.monthly_income = monthly_income
            account.email = email
            account.phone = phone
            account.address = address
            account.dob = dob
            account.add_transaction("Modification", 0, "Account details modified")
            self.save_accounts()
            return account
        return None

    def get_all_accounts(self):
        return self.accounts

    def get_account_balances(self):
        accNos = [a.accNo for a in self.accounts]
        balances = [a.deposit for a in self.accounts]
        return accNos, balances

    def calculate_emi(self, loan_amount, tenure, interest_rate=10.0):
        R = interest_rate / (12 * 100)
        N = tenure
        P = loan_amount
        EMI = (P * R * (1+R)**N) / ((1+R)**N - 1) if ((1+R)**N - 1) != 0 else 0
        return round(EMI, 2)

    def apply_loan(self, accNo, loan_amount, tenure, interest_rate=10.0):
        account = self.get_account(accNo)
        if account:
            EMI = self.calculate_emi(loan_amount, tenure, interest_rate)
            account.loan_details = {
                'loan_amount': loan_amount,
                'interest_rate': interest_rate,
                'tenure': tenure,
                'EMI': EMI,
                'outstanding_amount': loan_amount,
                'status': 'Active'
            }
            account.add_transaction("Loan Applied", loan_amount,
                                    f"Loan for {tenure} months at {interest_rate}% interest. EMI: {EMI}")
            self.save_accounts()
            return account.loan_details
        return None

    def early_loan_repayment(self, accNo, payment):
        account = self.get_account(accNo)
        if account and account.loan_details:
            if account.loan_details["status"] == "Paid Off":
                return "Loan already paid off"
            account.loan_details["outstanding_amount"] -= payment
            account.add_transaction("Loan Repayment", payment, "Early loan repayment")
            if account.loan_details["outstanding_amount"] <= 0:
                account.loan_details["status"] = "Paid Off"
                account.loan_details["outstanding_amount"] = 0
            self.save_accounts()
            return account.loan_details
        return "No loan found for account"

    def transfer_funds(self, from_acc, to_acc, amount):
        account_from = self.get_account(from_acc)
        account_to = self.get_account(to_acc)
        if account_from and account_to:
            if account_from.deposit >= amount:
                account_from.deposit -= amount
                account_to.deposit += amount
                account_from.add_transaction("Transfer Out", amount, f"Transferred to {to_acc}")
                account_to.add_transaction("Transfer In", amount, f"Received from {from_acc}")
                self.save_accounts()
                return True
            else:
                return "Insufficient Funds"
        return "Account not found"

    # Admin: KYC Approvals
    def approve_kyc(self, accNo):
        account = self.get_account(accNo)
        if account and account.kyc_status == "Under Review":
            account.kyc_status = "Approved"
            self.save_accounts()
            return True
        return False

    def reject_kyc(self, accNo):
        account = self.get_account(accNo)
        if account and account.kyc_status == "Under Review":
            account.kyc_status = "Rejected"
            self.save_accounts()
            return True
        return False

bank_system = BankSystem()

###############################################################################
# MUTUAL FUNDS INVESTMENT MODULE
###############################################################################
def sip_calculator(P, annual_rate, months):
    r = (annual_rate / 100) / 12
    FV = P * (((1 + r)**months - 1) / r) * (1 + r)
    return FV

def lump_sum_calculator(principal, annual_rate, years):
    r = annual_rate / 100
    FV = principal * ((1 + r)**years)
    return FV

def cagr_calculator(begin_value, end_value, years):
    if begin_value <= 0 or years <= 0:
        return None
    CAGR = (end_value / begin_value)**(1/years) - 1
    return CAGR

# SIP Calculator Widgets
sip_P = widgets.FloatText(description="Monthly Investment:")
sip_rate = widgets.FloatText(description="Annual Return (%):")
sip_months = widgets.IntText(description="Duration (months):")
sip_button = widgets.Button(description="Calculate SIP Future Value", button_style="success")
sip_output = widgets.Output()

def calculate_sip(b):
    with sip_output:
        clear_output()
        try:
            P = sip_P.value
            annual_rate = sip_rate.value
            months = sip_months.value
            FV = sip_calculator(P, annual_rate, months)
            print(f"SIP Future Value: {FV:.2f}")
        except Exception as e:
            print("Error:", e)

sip_button.on_click(calculate_sip)
sip_box = widgets.VBox([sip_P, sip_rate, sip_months, sip_button, sip_output])

# Lump Sum Calculator Widgets
lump_principal = widgets.FloatText(description="Principal:")
lump_rate = widgets.FloatText(description="Annual Return (%):")
lump_years = widgets.IntText(description="Duration (years):")
lump_button = widgets.Button(description="Calculate Lump Sum Future Value", button_style="success")
lump_output = widgets.Output()

def calculate_lump(b):
    with lump_output:
        clear_output()
        try:
            principal = lump_principal.value
            annual_rate = lump_rate.value
            years = lump_years.value
            FV = lump_sum_calculator(principal, annual_rate, years)
            print(f"Lump Sum Future Value: {FV:.2f}")
        except Exception as e:
            print("Error:", e)

lump_button.on_click(calculate_lump)
lump_box = widgets.VBox([lump_principal, lump_rate, lump_years, lump_button, lump_output])

# CAGR Calculator Widgets
cagr_begin = widgets.FloatText(description="Beginning Value:")
cagr_end = widgets.FloatText(description="Ending Value:")
cagr_years = widgets.IntText(description="Duration (years):")
cagr_button = widgets.Button(description="Calculate CAGR", button_style="success")
cagr_output = widgets.Output()

def calculate_cagr(b):
    with cagr_output:
        clear_output()
        try:
            begin_value = cagr_begin.value
            end_value = cagr_end.value
            years = cagr_years.value
            CAGR = cagr_calculator(begin_value, end_value, years)
            if CAGR is not None:
                print(f"CAGR: {CAGR*100:.2f}%")
            else:
                print("Invalid input for CAGR calculation.")
        except Exception as e:
            print("Error:", e)

cagr_button.on_click(calculate_cagr)
cagr_box = widgets.VBox([cagr_begin, cagr_end, cagr_years, cagr_button, cagr_output])

mutual_funds_tab = widgets.Tab(children=[sip_box, lump_box, cagr_box])
mutual_funds_tab.set_title(0, "SIP Calculator")
mutual_funds_tab.set_title(1, "Lump Sum Calculator")
mutual_funds_tab.set_title(2, "CAGR Calculator")

mutual_funds_intro = widgets.HTML("""
<h3>Mutual Funds Investment Module</h3>
<p>
Mutual funds allow individual investors to participate in a diversified portfolio of stocks, bonds, or other securities.
They offer professional management and flexible investment options to grow wealth and manage risk.
</p>
<h4>Key Features</h4>
<ul>
  <li><b>Diverse Fund Options:</b> Equity, Debt, Hybrid, and Index Funds</li>
  <li><b>Interactive Investment Calculators:</b> SIP, Lump Sum, and CAGR Calculators</li>
  <li><b>Performance & Risk Metrics:</b> NAV, expense ratios, volatility, Sharpe ratio, and benchmark comparisons</li>
  <li><b>Customized Reporting:</b> Detailed reports based on your inputs and historical data</li>
</ul>
""")
mutual_funds_module = widgets.VBox([mutual_funds_intro, mutual_funds_tab])

###############################################################################
# ADVANCED INVESTMENT TOOLS MODULE
###############################################################################
# 1. Interactive Portfolio Builder & Risk Profiling
risk_question1 = widgets.RadioButtons(options=["Short (<3 years)", "Medium (3-7 years)", "Long (>7 years)"],
                                        description="Investment Horizon:")
risk_question2 = widgets.RadioButtons(options=["Low", "Medium", "High"], description="Risk Tolerance:")
risk_question3 = widgets.RadioButtons(options=["Not important", "Important"], description="Liquidity Requirement:")
risk_button = widgets.Button(description="Submit Risk Profile", button_style="info")
risk_output = widgets.Output()

def risk_profile_callback(b):
    with risk_output:
        clear_output()
        horizon = risk_question1.value
        risk_tol = risk_question2.value
        liquidity = risk_question3.value
        if risk_tol == "Low":
            profile = "Conservative"
            allocation = "70% Debt, 30% Equity"
        elif risk_tol == "Medium":
            profile = "Balanced"
            allocation = "50% Debt, 50% Equity"
        else:
            profile = "Aggressive"
            allocation = "30% Debt, 70% Equity"
        print("Risk Profiling Result:")
        print(f"Investment Horizon: {horizon}")
        print(f"Risk Tolerance: {risk_tol}")
        print(f"Liquidity Requirement: {liquidity}")
        print(f"Recommended Profile: {profile}")
        print(f"Suggested Asset Allocation: {allocation}")

risk_button.on_click(risk_profile_callback)
risk_profile_box = widgets.VBox([risk_question1, risk_question2, risk_question3, risk_button, risk_output])

# 2. Advanced Data Visualization – Historical Performance & Comparisons
performance_button = widgets.Button(description="Show Historical Performance", button_style="info")
performance_output = widgets.Output()

def performance_callback(b):
    with performance_output:
        clear_output()
        dates = pd.date_range(end=datetime.today(), periods=50)
        fund1 = np.cumsum(np.random.randn(50)) + 100
        fund2 = np.cumsum(np.random.randn(50)) + 120
        df = pd.DataFrame({"Date": dates, "Fund A NAV": fund1, "Fund B NAV": fund2})
        fig = px.line(df, x="Date", y=["Fund A NAV", "Fund B NAV"],
                      title="Historical NAV Trends")
        fig.show()
performance_button.on_click(performance_callback)
performance_box = widgets.VBox([performance_button, performance_output])

# 3. Real-Time Market Updates (Simulated)
market_button2 = widgets.Button(description="Refresh Market Data", button_style="info")
market_output2 = widgets.Output()

def market_callback2(b):
    with market_output2:
        clear_output()
        nav_fundA = round(100 + random.uniform(-5, 5), 2)
        nav_fundB = round(120 + random.uniform(-5, 5), 2)
        news = random.choice([
            "Market rallies amid economic optimism.",
            "Global indices fall due to geopolitical tensions.",
            "Tech stocks soar on robust earnings.",
            "Bond yields rise; investors cautious."
        ])
        print(f"Fund A NAV: {nav_fundA}")
        print(f"Fund B NAV: {nav_fundB}")
        print("Latest Market News:", news)
market_button2.on_click(market_callback2)
market_box = widgets.VBox([market_button2, market_output2])

# 4. Additional Calculators & Tools
tax_amount = widgets.FloatText(description="Investment Amount:")
tax_rate = widgets.FloatText(description="Tax Rate (%):")
tax_button = widgets.Button(description="Calculate Tax Impact", button_style="success")
tax_output = widgets.Output()

def tax_calculator(b):
    with tax_output:
        clear_output()
        amount = tax_amount.value
        rate = tax_rate.value / 100
        tax = amount * rate
        print(f"Tax Impact: ${tax:.2f}")
tax_button.on_click(tax_calculator)
tax_box = widgets.VBox([tax_amount, tax_rate, tax_button, tax_output])

nominal_return = widgets.FloatText(description="Nominal Return (%):")
inflation_rate = widgets.FloatText(description="Inflation Rate (%):")
inflation_button = widgets.Button(description="Calculate Real Return", button_style="success")
inflation_output = widgets.Output()

def inflation_calculator(b):
    with inflation_output:
        clear_output()
        n = nominal_return.value / 100
        i = inflation_rate.value / 100
        real_return = ((1+n)/(1+i)) - 1
        print(f"Inflation-Adjusted (Real) Return: {real_return*100:.2f}%")
inflation_button.on_click(inflation_calculator)
inflation_box = widgets.VBox([nominal_return, inflation_rate, inflation_button, inflation_output])

rebalance_slider = widgets.IntSlider(value=12, min=1, max=24, description="Rebalance Frequency (months):")
rebalance_button = widgets.Button(description="Simulate Rebalancing Effect", button_style="success")
rebalance_output = widgets.Output()

def rebalancing_simulator(b):
    with rebalance_output:
        clear_output()
        freq = rebalance_slider.value
        effect = random.uniform(0, 10)
        print(f"Simulated improvement: {effect:.2f}% with rebalancing every {freq} months.")
rebalance_button.on_click(rebalancing_simulator)
rebalance_box = widgets.VBox([rebalance_slider, rebalance_button, rebalance_output])

calc_tools_tab = widgets.Tab(children=[tax_box, inflation_box, rebalance_box])
calc_tools_tab.set_title(0, "Tax Impact")
calc_tools_tab.set_title(1, "Real Return")
calc_tools_tab.set_title(2, "Rebalancing")

# 5. Community & Educational Content
faq_text = widgets.HTML("""
<h4>Glossary & FAQs</h4>
<ul>
  <li><b>NAV:</b> Net Asset Value - per unit market value of a fund.</li>
  <li><b>Expense Ratio:</b> Annual fee as a percentage of assets.</li>
  <li><b>Sharpe Ratio:</b> Measure of risk-adjusted return.</li>
  <li><b>CAGR:</b> Compound Annual Growth Rate.</li>
</ul>
""")
tutorial_text = widgets.HTML("""
<h4>Tutorials & Webinars</h4>
<p>Watch our video tutorials and join live webinars to learn how to choose the right mutual funds.</p>
""")
discussion_area = widgets.Textarea(value="Share your thoughts here...", description="Discussion:")
discussion_button = widgets.Button(description="Submit Comment", button_style="info")
discussion_output = widgets.Output()

def submit_comment(b):
    with discussion_output:
        clear_output()
        comment = discussion_area.value
        print("Your comment has been submitted:")
        print(comment)
discussion_button.on_click(submit_comment)
community_box = widgets.VBox([faq_text, tutorial_text, discussion_area, discussion_button, discussion_output])
edu_tab = widgets.Tab(children=[community_box])
edu_tab.set_title(0, "Glossary & Forums")

investment_tools_tab = widgets.Tab(children=[
    risk_profile_box,
    performance_box,
    market_box,
    calc_tools_tab,
    edu_tab
])
investment_tools_tab.set_title(0, "Portfolio Builder & Risk Profiling")
investment_tools_tab.set_title(1, "Historical Performance")
investment_tools_tab.set_title(2, "Real-Time Market Updates")
investment_tools_tab.set_title(3, "Additional Calculators")
investment_tools_tab.set_title(4, "Community & Education")

advanced_investment_tools_module = widgets.VBox([
    widgets.HTML("""
    <h2>Advanced Investment Tools</h2>
    <p>
      Explore a suite of advanced tools to build your portfolio and analyze mutual fund investments.
    </p>
    """),
    investment_tools_tab
])

###############################################################################
# MODERN MAP INTEGRATION USING FOLIUM (MAHARASHTRA, INDIA)
###############################################################################
gm_output = widgets.Output()
gm_location = widgets.Text(description="Location:", style={'description_width':'initial'}, placeholder="Enter a location (default: Maharashtra)")
gm_button = widgets.Button(description="Show Map", button_style='info', icon='map-marker')

def show_map_callback(b):
    with gm_output:
        clear_output()
        loc = gm_location.value.strip() if gm_location.value.strip() else "Maharashtra, India"
        if "maharashtra" in loc.lower():
            lat, lon = 19.7515, 75.7139
        else:
            lat, lon = 20.5937, 78.9629
        m = folium.Map(location=[lat, lon], zoom_start=7)
        folium.Marker(location=[lat, lon], popup=loc).add_to(m)
        m.save("maharashtra_map.html")
        display(IFrame(src="maharashtra_map.html", width="600", height="450"))

gm_button.on_click(show_map_callback)
google_map_tab = widgets.VBox([
    widgets.HTML("<h4>Modern Map Integration (Folium)</h4>"),
    gm_location,
    gm_button,
    gm_output
])

###############################################################################
# EXTRA MODULES
###############################################################################
# (1) Upskilling & Education
ed_output = widgets.Output()
ed_button = widgets.Button(description="Show Ed Resources", button_style='info')
ed_button.on_click(ed_callback)
education_tab = widgets.VBox([
    widgets.Label("Upskilling & Financial Education"),
    ed_button,
    ed_output
])

# (2) Loan Marketplace
market_output = widgets.Output()
market_acc_input = widgets.IntText(description="Acc No:")
market_button = widgets.Button(description="Show Loan Offers", button_style='success')
market_button.on_click(market_callback)
marketplace_tab = widgets.VBox([
    widgets.Label("Personalized Credit & Loan Marketplace"),
    market_acc_input,
    market_button,
    market_output
])

# (3) Digital Wallet
wallet_output = widgets.Output()
wallet_button = widgets.Button(description="Create Virtual Card", button_style='info')
wallet_button.on_click(wallet_callback)
wallet_tab = widgets.VBox([
    widgets.Label("Digital Wallet & Virtual Cards"),
    wallet_button,
    wallet_output
])

# (4) Family & Group Savings
family_output = widgets.Output()
fam_goal_input = widgets.Text(description="Goal Name:")
fam_amt_input = widgets.IntText(description="Goal Amt:")
fam_button = widgets.Button(description="Create Goal", button_style="info")
fam_button.on_click(fam_callback)
family_tab = widgets.VBox([
    widgets.Label("Family & Group Savings Goals"),
    fam_goal_input,
    fam_amt_input,
    fam_button,
    family_output
])

# (5) Support & Help
support_output = widgets.Output()
support_button = widgets.Button(description="Contact Support", button_style='warning')
support_button.on_click(support_callback)
support_tab = widgets.VBox([
    widgets.Label("Customer Support & Help Center"),
    support_button,
    support_output
])

# (6) Unified Login
unified_output = widgets.Output()
unified_label = widgets.Label("Unified Login for Web & Mobile: Single Sign-On (Placeholder).")
unified_tab = widgets.VBox([
    unified_label,
    unified_output
])

# (7) e-Statements Document Management
statements_2 = widgets.VBox([widgets.Label("Already integrated with e-Statements tab.")])

# (8) Scalable Architecture
scalable_output = widgets.Output()
scalable_label = widgets.Label("Discussion about horizontal scaling, load balancing, etc. (Placeholder).")
scalable_tab = widgets.VBox([
    scalable_label,
    scalable_output
])

extra_modules_tab = widgets.Tab(children=[
    support_tab,
    unified_tab,
    statements_2,
    scalable_tab
])
for i, t in enumerate(["Support/Help", "Unified Login", "e-Statements", "Scalable Arch"]):
    extra_modules_tab.set_title(i, t)

final_new_modules_tab = widgets.Tab(children=[
    education_tab,
    marketplace_tab,
    wallet_tab,
    family_tab,
    extra_modules_tab
])
final_titles = [
    "Upskill & Education",
    "Loan Marketplace",
    "Digital Wallet",
    "Family Savings",
    "More Modules"
]
for i, t in enumerate(final_titles):
    final_new_modules_tab.set_title(i, t)

###############################################################################
# Extra Advanced Modules
###############################################################################
# A) Advanced ML Extension using XGBoost Tuning (Already integrated above)
advanced_ml_ext_tab = advanced_ml_tab  # Reuse advanced_ml_tab

# B) Enhanced Admin Features (Extended)
enh_admin_output = widgets.Output()
enh_admin_button = widgets.Button(description="Show Recent Transactions", button_style="warning")
def enh_admin_button_callback(b):
    with enh_admin_output:
        clear_output()
        all_txns = []
        for acc in bank_system.get_all_accounts():
            all_txns.extend(acc.transaction_history)
        all_txns.sort(key=lambda txn: datetime.strptime(txn['timestamp'], "%Y-%m-%d %H:%M:%S"), reverse=True)
        recent = all_txns[:10]
        if recent:
            df = pd.DataFrame(recent)
            print("Recent Transactions Across All Accounts:")
            display(df)
        else:
            print("No transactions found.")
enh_admin_button.on_click(enh_admin_button_callback)
enhanced_admin_tab = widgets.VBox([
    widgets.HTML("<h4>Enhanced Admin Features</h4>"),
    refresh_admin_button,
    enh_admin_button,
    enh_admin_output
])

# C) Enhanced User Experience (Theme Toggle)
eux_output = widgets.Output()
theme_dropdown = widgets.Dropdown(options=["Light", "Dark", "Solarized"], description="Select Theme:")
def apply_theme(theme):
    if theme == "Light":
        dashboard_accordion.layout = widgets.Layout(background_color="white", color="black")
    elif theme == "Dark":
        dashboard_accordion.layout = widgets.Layout(background_color="black", color="white")
    elif theme == "Solarized":
        dashboard_accordion.layout = widgets.Layout(background_color="#FDF6E3", color="#657B83")
    else:
        dashboard_accordion.layout = widgets.Layout(background_color="white", color="black")
    print(f"{theme} theme applied.")
def theme_change_callback(change):
    apply_theme(change['new'])
theme_dropdown.observe(theme_change_callback, names='value')
enhanced_user_experience_tab = widgets.VBox([
    widgets.HTML("<h4>Enhanced User Experience</h4>"),
    theme_dropdown,
    eux_output
])
with eux_output:
    clear_output()
    print("Select a theme to apply.")

# D) Advanced Analytics and Reporting (Extended)
aar_output = widgets.Output()
aar_button = widgets.Button(description="Show Advanced Analytics", button_style="info")
def aar_button_callback(b):
    with aar_output:
        clear_output()
        accounts = bank_system.get_all_accounts()
        if not accounts:
            print("No account data available.")
            return
        data = {
            "AccNo": [a.accNo for a in accounts],
            "Deposit": [a.deposit for a in accounts],
            "MonthlyIncome": [a.monthly_income for a in accounts],
            "Transactions": [len(a.transaction_history) for a in accounts]
        }
        df = pd.DataFrame(data)
        print("Overall Account Data:")
        display(df)
        fig1 = px.scatter(df, x="MonthlyIncome", y="Deposit", size="Transactions",
                          title="Monthly Income vs Deposit")
        fig1.show()
        fig2 = px.histogram(df, x="Deposit", nbins=20, title="Deposit Distribution")
        fig2.show()
        corr = df[["Deposit", "MonthlyIncome", "Transactions"]].corr()
        plt.figure(figsize=(5,4))
        sns.heatmap(corr, annot=True, cmap="coolwarm")
        plt.title("Correlation Heatmap")
        plt.show()
aar_button.on_click(aar_button_callback)
advanced_analytics_reporting_tab = widgets.VBox([
    widgets.HTML("<h4>Advanced Analytics & Reporting</h4>"),
    aar_button,
    aar_output
])

# E) Personal Financial Management (Extended)
pfm_output = widgets.Output()
pfm_acc_input = widgets.IntText(description="Acc No:")
pfm_button = widgets.Button(description="View Financial Summary", button_style="success")
def pfm_button_callback(b):
    with pfm_output:
        clear_output()
        accNo = pfm_acc_input.value
        acct = bank_system.get_account(accNo)
        if not acct:
            print("Account not found!")
            return
        withdrawals = [txn for txn in acct.transaction_history if "Withdrawal" in txn['type']]
        if not withdrawals:
            print("No withdrawal transactions found.")
            return
        total_expense = sum(txn['amount'] for txn in withdrawals)
        dates = [datetime.strptime(txn['timestamp'], "%Y-%m-%d %H:%M:%S") for txn in withdrawals]
        period_days = (max(dates) - min(dates)).days or 1
        monthly_expense = total_expense / period_days * 30
        print(f"Estimated Average Monthly Expense for Acc {accNo}: ${monthly_expense:.2f}")
        savings_target = acct.monthly_income * 0.2
        print(f"Suggested Monthly Savings Target: ${savings_target:.2f}")
        categories = ["Food", "Rent", "Utilities", "Entertainment", "Others"]
        expenses = np.random.randint(50, 500, size=len(categories))
        plt.figure(figsize=(5,5))
        plt.pie(expenses, labels=categories, autopct='%1.1f%%', startangle=90)
        plt.title("Expense Category Breakdown (Simulated)")
        plt.axis('equal')
        plt.show()
pfm_button.on_click(pfm_button_callback)
personal_financial_management_tab = widgets.VBox([
    widgets.HTML("<h4>Personal Financial Management</h4>"),
    pfm_acc_input,
    pfm_button,
    pfm_output
])

###############################################################################
# Assemble Advanced Investment Tools Module
###############################################################################
# 1. Interactive Portfolio Builder & Risk Profiling
risk_profile_box = widgets.VBox([risk_question1, risk_question2, risk_question3, risk_button, risk_output])
# 2. Historical Performance & Comparisons
performance_box = widgets.VBox([performance_button, performance_output])
# 3. Real-Time Market Updates is market_box (already defined)
# 4. Additional Calculators is calc_tools_tab (already defined)
# 5. Community & Education is edu_tab (already defined)
investment_tools_tab = widgets.Tab(children=[
    risk_profile_box,
    performance_box,
    market_box,
    calc_tools_tab,
    edu_tab
])
investment_tools_tab.set_title(0, "Portfolio Builder & Risk Profiling")
investment_tools_tab.set_title(1, "Historical Performance")
investment_tools_tab.set_title(2, "Real-Time Market Updates")
investment_tools_tab.set_title(3, "Additional Calculators")
investment_tools_tab.set_title(4, "Community & Education")
advanced_investment_tools_module = widgets.VBox([
    widgets.HTML("""
    <h2>Advanced Investment Tools</h2>
    <p>Dive deeper into your mutual fund investments with our advanced tools:</p>
    <ul>
      <li><b>Interactive Portfolio Builder & Risk Profiling:</b> Answer questions to determine your risk tolerance and get asset allocation suggestions.</li>
      <li><b>Historical Performance & Comparisons:</b> View interactive charts showing historical NAV trends and compare funds.</li>
      <li><b>Real-Time Market Updates:</b> Get simulated live updates on fund performance and market news.</li>
      <li><b>Additional Calculators & Tools:</b> Calculate tax impact, inflation-adjusted returns, and simulate rebalancing effects.</li>
      <li><b>Community & Educational Content:</b> Access FAQs, tutorials, and participate in discussion forums.</li>
    </ul>
    """),
    investment_tools_tab
])

###############################################################################
# MODERN MAP INTEGRATION USING FOLIUM (MAHARASHTRA, INDIA)
###############################################################################
gm_output = widgets.Output()
gm_location = widgets.Text(description="Location:", style={'description_width':'initial'}, placeholder="Enter a location (default: Maharashtra)")
gm_button = widgets.Button(description="Show Map", button_style='info', icon='map-marker')

def show_map_callback(b):
    with gm_output:
        clear_output()
        loc = gm_location.value.strip() if gm_location.value.strip() else "Maharashtra, India"
        if "maharashtra" in loc.lower():
            lat, lon = 19.7515, 75.7139
        else:
            lat, lon = 20.5937, 78.9629
        m = folium.Map(location=[lat, lon], zoom_start=7)
        folium.Marker(location=[lat, lon], popup=loc).add_to(m)
        m.save("maharashtra_map.html")
        display(IFrame(src="maharashtra_map.html", width="600", height="450"))

gm_button.on_click(show_map_callback)
google_map_tab = widgets.VBox([
    widgets.HTML("<h4>Modern Map Integration (Folium)</h4>"),
    gm_location,
    gm_button,
    gm_output
])

###############################################################################
# EXTRA MODULES
###############################################################################
# (1) Upskilling & Education
ed_output = widgets.Output()
ed_button = widgets.Button(description="Show Ed Resources", button_style='info')
ed_button.on_click(ed_callback)
education_tab = widgets.VBox([
    widgets.Label("Upskilling & Financial Education"),
    ed_button,
    ed_output
])

# (2) Loan Marketplace
market_output = widgets.Output()
market_acc_input = widgets.IntText(description="Acc No:")
market_button = widgets.Button(description="Show Loan Offers", button_style='success')
market_button.on_click(market_callback)
marketplace_tab = widgets.VBox([
    widgets.Label("Personalized Credit & Loan Marketplace"),
    market_acc_input,
    market_button,
    market_output
])

# (3) Digital Wallet
wallet_output = widgets.Output()
wallet_button = widgets.Button(description="Create Virtual Card", button_style='info')
wallet_button.on_click(wallet_callback)
wallet_tab = widgets.VBox([
    widgets.Label("Digital Wallet & Virtual Cards"),
    wallet_button,
    wallet_output
])

# (4) Family & Group Savings
family_output = widgets.Output()
fam_goal_input = widgets.Text(description="Goal Name:")
fam_amt_input = widgets.IntText(description="Goal Amt:")
fam_button = widgets.Button(description="Create Goal", button_style="info")
fam_button.on_click(fam_callback)
family_tab = widgets.VBox([
    widgets.Label("Family & Group Savings Goals"),
    fam_goal_input,
    fam_amt_input,
    fam_button,
    family_output
])

# (5) Support & Help
support_output = widgets.Output()
support_button = widgets.Button(description="Contact Support", button_style='warning')
support_button.on_click(support_callback)
support_tab = widgets.VBox([
    widgets.Label("Customer Support & Help Center"),
    support_button,
    support_output
])

# (6) Unified Login
unified_output = widgets.Output()
unified_label = widgets.Label("Unified Login for Web & Mobile: Single Sign-On (Placeholder).")
unified_tab = widgets.VBox([
    unified_label,
    unified_output
])

# (7) e-Statements Document Management
statements_2 = widgets.VBox([widgets.Label("Already integrated with e-Statements tab.")])

# (8) Scalable Architecture
scalable_output = widgets.Output()
scalable_label = widgets.Label("Discussion about horizontal scaling, load balancing, etc. (Placeholder).")
scalable_tab = widgets.VBox([
    scalable_label,
    scalable_output
])

extra_modules_tab = widgets.Tab(children=[
    support_tab,
    unified_tab,
    statements_2,
    scalable_tab
])
for i, t in enumerate(["Support/Help", "Unified Login", "e-Statements", "Scalable Arch"]):
    extra_modules_tab.set_title(i, t)

final_new_modules_tab = widgets.Tab(children=[
    education_tab,
    marketplace_tab,
    wallet_tab,
    family_tab,
    extra_modules_tab
])
final_titles = [
    "Upskill & Education",
    "Loan Marketplace",
    "Digital Wallet",
    "Family Savings",
    "More Modules"
]
for i, t in enumerate(final_titles):
    final_new_modules_tab.set_title(i, t)

###############################################################################
# Extra Advanced Modules
###############################################################################
# A) Advanced ML Extension using XGBoost Tuning (Already integrated above)
advanced_ml_ext_tab = advanced_ml_tab

# B) Enhanced Admin Features (Extended)
enh_admin_output = widgets.Output()
enh_admin_button = widgets.Button(description="Show Recent Transactions", button_style="warning")
def enh_admin_button_callback(b):
    with enh_admin_output:
        clear_output()
        all_txns = []
        for acc in bank_system.get_all_accounts():
            all_txns.extend(acc.transaction_history)
        all_txns.sort(key=lambda txn: datetime.strptime(txn['timestamp'], "%Y-%m-%d %H:%M:%S"), reverse=True)
        recent = all_txns[:10]
        if recent:
            df = pd.DataFrame(recent)
            print("Recent Transactions Across All Accounts:")
            display(df)
        else:
            print("No transactions found.")
enh_admin_button.on_click(enh_admin_button_callback)
enhanced_admin_tab = widgets.VBox([
    widgets.HTML("<h4>Enhanced Admin Features</h4>"),
    refresh_admin_button,
    enh_admin_button,
    enh_admin_output
])

# C) Enhanced User Experience (Theme Toggle)
eux_output = widgets.Output()
theme_dropdown = widgets.Dropdown(options=["Light", "Dark", "Solarized"], description="Select Theme:")
def apply_theme(theme):
    if theme == "Light":
        dashboard_accordion.layout = widgets.Layout(background_color="white", color="black")
    elif theme == "Dark":
        dashboard_accordion.layout = widgets.Layout(background_color="black", color="white")
    elif theme == "Solarized":
        dashboard_accordion.layout = widgets.Layout(background_color="#FDF6E3", color="#657B83")
    else:
        dashboard_accordion.layout = widgets.Layout(background_color="white", color="black")
    print(f"{theme} theme applied.")
def theme_change_callback(change):
    apply_theme(change['new'])
theme_dropdown.observe(theme_change_callback, names='value')
enhanced_user_experience_tab = widgets.VBox([
    widgets.HTML("<h4>Enhanced User Experience</h4>"),
    theme_dropdown,
    eux_output
])
with eux_output:
    clear_output()
    print("Select a theme to apply.")

# D) Advanced Analytics and Reporting (Extended)
aar_output = widgets.Output()
aar_button = widgets.Button(description="Show Advanced Analytics", button_style="info")
def aar_button_callback(b):
    with aar_output:
        clear_output()
        accounts = bank_system.get_all_accounts()
        if not accounts:
            print("No account data available.")
            return
        data = {
            "AccNo": [a.accNo for a in accounts],
            "Deposit": [a.deposit for a in accounts],
            "MonthlyIncome": [a.monthly_income for a in accounts],
            "Transactions": [len(a.transaction_history) for a in accounts]
        }
        df = pd.DataFrame(data)
        print("Overall Account Data:")
        display(df)
        fig1 = px.scatter(df, x="MonthlyIncome", y="Deposit", size="Transactions",
                          title="Monthly Income vs Deposit")
        fig1.show()
        fig2 = px.histogram(df, x="Deposit", nbins=20, title="Deposit Distribution")
        fig2.show()
        corr = df[["Deposit", "MonthlyIncome", "Transactions"]].corr()
        plt.figure(figsize=(5,4))
        sns.heatmap(corr, annot=True, cmap="coolwarm")
        plt.title("Correlation Heatmap")
        plt.show()
aar_button.on_click(aar_button_callback)
advanced_analytics_reporting_tab = widgets.VBox([
    widgets.HTML("<h4>Advanced Analytics & Reporting</h4>"),
    aar_button,
    aar_output
])

# E) Personal Financial Management (Extended)
pfm_output = widgets.Output()
pfm_acc_input = widgets.IntText(description="Acc No:")
pfm_button = widgets.Button(description="View Financial Summary", button_style="success")
def pfm_button_callback(b):
    with pfm_output:
        clear_output()
        accNo = pfm_acc_input.value
        acct = bank_system.get_account(accNo)
        if not acct:
            print("Account not found!")
            return
        withdrawals = [txn for txn in acct.transaction_history if "Withdrawal" in txn['type']]
        if not withdrawals:
            print("No withdrawal transactions found.")
            return
        total_expense = sum(txn['amount'] for txn in withdrawals)
        dates = [datetime.strptime(txn['timestamp'], "%Y-%m-%d %H:%M:%S") for txn in withdrawals]
        period_days = (max(dates) - min(dates)).days or 1
        monthly_expense = total_expense / period_days * 30
        print(f"Estimated Average Monthly Expense for Acc {accNo}: ${monthly_expense:.2f}")
        savings_target = acct.monthly_income * 0.2
        print(f"Suggested Monthly Savings Target: ${savings_target:.2f}")
        categories = ["Food", "Rent", "Utilities", "Entertainment", "Others"]
        expenses = np.random.randint(50, 500, size=len(categories))
        plt.figure(figsize=(5,5))
        plt.pie(expenses, labels=categories, autopct='%1.1f%%', startangle=90)
        plt.title("Expense Category Breakdown (Simulated)")
        plt.axis('equal')
        plt.show()
pfm_button.on_click(pfm_button_callback)
personal_financial_management_tab = widgets.VBox([
    widgets.HTML("<h4>Personal Financial Management</h4>"),
    pfm_acc_input,
    pfm_button,
    pfm_output
])

###############################################################################
# Assemble Final Dashboard Accordion
###############################################################################
final_accordion_children = [
    account_mgmt_tab,
    transactions_tab,
    pie_chart_dashboard_tab,
    advanced_visualizations_tab,
    bank_names_tab,
    reports_tab,
    get_admin_panel_widget(),  # Admin Panel
    loan_application_tab,
    password_loan_management_tab,
    ml_tab,
    enhancements_tab,
    google_map_tab,
    final_new_modules_tab,
    advanced_ml_ext_tab,
    enhanced_admin_tab,
    enhanced_user_experience_tab,
    advanced_analytics_reporting_tab,
    personal_financial_management_tab,
    mutual_funds_module,
    advanced_investment_tools_module  # Advanced Investment Tools Module
]
final_accordion_titles = [
    "Account Management",
    "Transactions",
    "Pie Chart Dashboard",
    "Advanced Visualizations",
    "Bank Names by Location",
    "Reports & Exports",
    "Admin Panel",
    "Loan Application",
    "Password & Loan Management",
    "Machine Learning Models",
    "Enhancements",
    "Modern Map",
    "New Modules",
    "Advanced ML Extension",
    "Enhanced Admin",
    "Enhanced UX",
    "Advanced Analytics",
    "Personal Finance",
    "Mutual Funds Investment",
    "Advanced Investment Tools"
]

dashboard_accordion = widgets.Accordion(children=final_accordion_children)
for i, t in enumerate(final_accordion_titles):
    dashboard_accordion.set_title(i, t)

###############################################################################
# DISPLAY THE ENTIRE APP
###############################################################################
display(dashboard_accordion)


Accordion(children=(Tab(children=(VBox(children=(HTML(value='<h4>Secure Login / Logout</h4>'), IntText(value=0…